In [ ]:
from collections import Counter
import numpy as np
from visualize_events.snomed import *
from visualize_events.data import *
from visualize_events.algorithms import *
import matplotlib.pyplot as plt
import pandas as pd
import pydot
import networkx as nx
import numpy as np
from networkx.drawing.nx_agraph import graphviz_layout
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances
import plotly.graph_objects as go
import plotly
from shapely.ops import voronoi_diagram
from shapely.geometry import MultiPoint, Polygon, Point
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import Voronoi, voronoi_plot_2d
from plotly.subplots import make_subplots


T = load_dag()
print(len(T))

In [ ]:
#PRECOMPUTE DISTANCE BETWEEN EVERY AE (just too much man, 184. GiB alone to store the result...)
# S = T.keep_clinical_findings_and_body()
# len(S)
# S.add_parent_store()
# nodes = S.nodes.values()
# N = len(nodes)
# S.root._attr_ancs()
# anc_store = []
# for i, n in enumerate(nodes):
#     if i%10000==0: print(i/len(nodes))
#     anc_store.append(n.leafs)
# dist = np.zeros((N, N))
# for i, anc1 in enumerate(anc_store):
#     print(i)
#     for j, anc2 in enumerate(anc_store[i+1:], i+1):
# #         common = anc1.intersection(anc2)
#         diff = anc1.symmetric_difference(anc2)
#         #d = 1/len(common)
# #         d = 1/(sum(a.depth for a in common)+1)
#         d = len(diff)
#         dist[i, j] = d
#         dist[j, i] = d

In [ ]:
#load_data('facts')['tox_drug_id'].unique().tolist()

In [ ]:
preds = get_predictions(2793, predictor='ci_low') # frequency, ci_low, ci_upp
P = T.set_predictions(preds, aggregate='sum')
print(len(P))
# 2793 

In [ ]:
C = P.compact_preds()
C = C.set_predictions(preds, aggregate='sum')
C.descendant_pred()
len(C)

In [ ]:
from visualize_events import HexMerge
alg = HexMerge()
alg.fit(C)
fig = alg.make_fig()

In [ ]:
fig

In [ ]:
t = C
t.add_parent_store()
from itertools import combinations

t.descendant_pred()
pred_nodes = t.root.preds
weights = dict()
for n in t.nodes.values():
    if n is t.root:
        continue
    for a, b in combinations(n.preds, 2):
        if id(a) > id(b):
            a, b = b, a
        w = weights.get((a, b), 0)
        weights[(a, b)] = w + 1 * n.depth
print(weights)

def nx_from_edge_weights(edge_weights):
    if isinstance(edge_weights, dict):
        edge_weights = edge_weights.items()
    G = nx.Graph()
    G.add_edges_from([(*key, {"weight": weight}) for key, weight in edge_weights])
    return G

# Could check if entire thing is planar or do binary/exponential search backwards
sort_weights = sorted(weights.items(), key=lambda x: x[1], reverse=True)
include = sort_weights[:4].copy()
g = nx_from_edge_weights(include)
deleted = 0
added = 4
for key, weight in sort_weights[4:]:
    g.add_edge(*key, weight=weight)
    is_planar, emb = nx.check_planarity(g)
    if not is_planar:
        g.remove_edge(*key)
        deleted += 1
        if deleted == 100:
            break
    else:
        added += 1
    
print(f"Deleted {deleted} from {len(weights)} edges, added {added}")
pos = nx.planar_layout(g)
nx.draw(g, pos=pos)

In [ ]:
M = None # Planar subdivision
G_M = None # (weak) dual of M (planar triangulation)

# Triangulate G_M (or it already is)
# G_M has integer weighted nodes
# drawing is a configuration C(v) of w(v) tiles for each vertex v
# Guiding shapes.

# --- Compute mosaic drawing
# --- move and reshape
# --- correct sizes of configurations


In [ ]:
nodes = list(C.nodes.values())
N = len(C)
indx_P_nodes, P_nodes = [], []
for i, node in enumerate(nodes):
    if (node.pred is not None and node.pred != 0):
        indx_P_nodes.append(i)
        P_nodes.append(node)

# ------- CREATE CONSTRAINTS --------

depths = C.get_depths()

area_to_colors = []
for d in range(len(depths)):
    area_to_color = {n: [] for n in P_nodes}
    V_d = depths[d]
    for i, v in enumerate(V_d):
        Rv = v.preds
        for l in Rv:
            area_to_color[l].append(i)
    k = len(V_d)
    for key,value in area_to_color.items():
        if len(value) == 0:
            value.append(k)
            k+=1
    area_to_colors.append(area_to_color)


# ------- INITIAL POINT ASSIGNMENT --------
anc_store = C.get_ancestors()

dist = np.zeros((N, N))
for i, anc1 in enumerate(anc_store):
    for j, anc2 in enumerate(anc_store[i+1:], i+1):
#         common = anc1.intersection(anc2)
        diff = anc1.symmetric_difference(anc2)
        #d = 1/len(common)
#         d = 1/(sum(a.depth for a in common)+1)
        d = len(diff) / (len(anc1) + len(anc2))
        dist[i, j] = d
        dist[j, i] = d


similarity = dist[indx_P_nodes, :][:, indx_P_nodes]

tsne = TSNE(metric="precomputed",
            perplexity=30,
            early_exaggeration=10)
embedding = tsne.fit_transform(similarity)

scaler = MinMaxScaler(feature_range=(-1,1))
embedding = scaler.fit_transform(embedding)

# ------- IMPROVING POINT ASSIGNMENT --------
drag = 0.01
dt = 0.001
dt_decay = 0.999
dt_lower = 0.001
contribution_pw_repel = 1
contribution_edge_repel = 1

total = contribution_pw_repel + contribution_edge_repel
contribution_pw_repel /= total
contribution_edge_repel /= total

weights = np.array([p.pred for p in P_nodes])
cross = np.power(weights.reshape(-1, 1)@weights.reshape(1, -1), 1.5)

N_points = len(embedding)
# move points away from eachother
speed = np.zeros(embedding.shape)
for epoch in range(10):
    dt = dt * dt_decay
    if dt < dt_lower:
        print("Reached dt_lower, stopping")
        break
#     within = (embedding[:, 0] < 1) & (embedding[:, 0] > -1) & (embedding[:, 1] < 1) & (embedding[:, 1] > -1)
#     print(f"Inside: {np.mean(within) * 100}%")
    
    pw = pairwise_distances(embedding)
    for i in range(N_points):
        pw[i,i] = np.inf
    min_pw = np.min(pw)
#     print(f"Smallest pw: {min_pw}")
    indx_nonzero = (pw == 0).nonzero()
    for i, j in zip(*indx_nonzero):
        embedding[i] += np.random.randn(2)
        pw[i,j] = np.inf
    min_pw = np.min(pw)

    # pairwise repelling
    # F = 1/d^2 * (u-v)/|u-v|
    F_pw = np.divide(cross, pw * pw * pw) # TODO base this on pairwise weight!
    assert F_pw.shape == pw.shape 
    move = np.tile(embedding.reshape(N_points, 1, 2), (1, N_points, 1)) - np.tile(embedding.reshape(1, N_points, 2), (N_points, 1, 1))
    
    F_pw = np.tile(F_pw.reshape(N_points, N_points, 1), (1, 1, 2)) * move
    F_pw = np.sum(F_pw, axis=1)

    
    # repelling from edges
    # direction: center, magnitude: ~ L5 norm ( not L2 so corners can get fuller )
    # also cut off repelling at certain ratio
    F_edge = np.zeros(embedding.shape)
    magnitude = np.linalg.norm(embedding, ord=10, axis=1)
    min_mag = 0.8
    magnitude = magnitude * (magnitude > min_mag)
    F_edge[:, 0] = (-embedding[:, 0]) * magnitude
    F_edge[:, 1] = (-embedding[:, 1]) * magnitude
    
    # attracting based on similarity
    # attracting based on visualization???

    F_pw_mean = np.mean(np.linalg.norm(F_pw, axis=1))
    F_pw = F_pw / F_pw_mean
    F_edge_mean = np.mean(np.linalg.norm(F_edge, axis=1))
    F_edge = F_edge / F_edge_mean
    F_sum = F_pw * contribution_pw_repel + F_edge * contribution_edge_repel

    # Hi
    speed += F_sum * dt
    
    speed -= speed * drag
    embedding += speed * dt

within = (embedding[:, 0] < 1) & (embedding[:, 0] > -1) & (embedding[:, 1] < 1) & (embedding[:, 1] > -1)
print(f"Inside: {np.mean(within) * 100}%")
# ------- COMPUTE VORONOI --------
points = MultiPoint(embedding.tolist())

regions = voronoi_diagram(points)

min_x = -1
min_y = -1
max_x = 1
max_y = 1
box = Polygon([[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]])

P_to_region = {}
for i, node in enumerate(P_nodes):
    coord = embedding[i]
    p = Point(*coord)
    found = False
    for region in regions:
        if region.contains(p):
            assert not found
            found = True
            P_to_region[node] = region

# ------- PLOT --------

def path(coords):
    svg = "M" + "L".join([str(px) + "," + str(py) for px,py in coords]) + "Z" # could do coords[:-1] I think
    return svg



In [ ]:
depth_choices = range(len(depths))#[0,1,2,3,4,5,10,15,20]
figs = []
for subplot_i, d in enumerate(depth_choices):
    fig = go.Figure()

    # Update axes properties
    fig.update_xaxes(
        showticklabels=False,
        showgrid=False,
        zeroline=False,
    )

    fig.update_yaxes(
        showticklabels=False,
        showgrid=False,
        zeroline=False,
        scaleanchor = "x",
        scaleratio = 1,
    )

    shapes = []
    colors = plotly.colors.sample_colorscale(colorscale='Turbo', samplepoints=k)
    np.random.shuffle(colors)
    for i, node_with_pred in enumerate(P_nodes):
        region = P_to_region[node_with_pred]
        poly = region.intersection(box)
        polygon = [p for p in poly.exterior.coords]
    #     print(polygon)
    #     plt.fill(*zip(*polygon), alpha=0.4)

        assignments = area_to_colors[d][node_with_pred]
        color = colors[np.random.choice(assignments).item()]
        shapes.append(
            dict(
                type="path",
                path=path(polygon),
                fillcolor=color,
                line_color=color,
            )
        )


    # Add shapes
    fig.update_layout(
        shapes=shapes,
        xaxis_range=[-1,1],
        yaxis_range=[-1,1],
        hovermode="closest"
    )

    figs.append(fig)

for depth, fig in zip(depth_choices, figs):
    print("Depth of ", depth)
    fig.show()

In [ ]:
embedding

In [ ]:
import matplotlib.pyplot as plt
fig = voronoi_plot_2d(vor)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pydot
import networkx as nx
import numpy as np
from networkx.drawing.nx_agraph import graphviz_layout
from sklearn.manifold import TSNE

In [ ]:
tree = C.copy(init=False)
tree = tree.set_predictions(preds)
tree.add_parent_store()
print(len(tree))
# tree.splice(keep_constraint = lambda n: n.pred_agg >= 0.01)

tree.attr_label()
edges = [
    (node.name, child.name) for node in tree.nodes.values() for child in node.children
]
g = nx.DiGraph()
g.add_edges_from(edges)
nodes = list(tree.nodes.values())
#draw_nodes = [n for n in nodes if n.pred_agg > 0.01]
# draw_nodes = nodes

N = len(nodes)

anc_store = tree.get_ancestors()

dist = np.zeros((N, N))
for i, anc1 in enumerate(anc_store):
    for j, anc2 in enumerate(anc_store[i+1:], i+1):
#         common = anc1.intersection(anc2)
        diff = anc1.symmetric_difference(anc2)
        #d = 1/len(common)
#         d = 1/(sum(a.depth for a in common)+1)
        d = len(diff)
        dist[i, j] = d
        dist[j, i] = d

print("computing draw nodes")
# draw_nodes = nodes
# draw_nodes = representative(tree, 50, score=lambda w, d: w * 1/(1.9**d))
draw_nodes = representative(tree, 50, score=lambda w, d: w * 1/(1+d))
draw_nodes = greedy_plus(tree, 50)
print("draw nodes computed")
        
tsne = TSNE(metric="precomputed",
            perplexity=30,
            early_exaggeration=10)
embedding = tsne.fit_transform(dist)
pos = {n.name: tuple(embedding[i]) for i, n in enumerate(nodes) if n in nodes}

scale = max(p[0] for p in pos.values()) - min(p[0] for p in pos.values())

fig = plt.figure(figsize=(15,15))
ax = fig.gca()

SCALE = 10
AGG_FAC = 1.

f_pred = scale * SCALE
f_agg = f_pred * AGG_FAC

def node_size(node):
    s = 1
    if node.pred is not None:
        s += node.pred * f_pred
    elif node.pred_agg is not None:
        s += node.pred_agg * f_agg
    return s

def node_color(node):
    if node.pred is not None:
        return "red"
    else:
        return "blue"

draw_nodes_names = [n.name for n in draw_nodes]
# edges = [edge for edge in g.edges() if (edge[0] in draw_nodes_names and edge[1] in draw_nodes_names)]
edges = []
# for name, node in tree.nodes.items():
#     for child in node.children:
#         c_name = child.name
#         edges.append((name, c_name))

width = []
for edge in edges:
    a = pos[edge[0]]
    b = pos[edge[1]]
    dist = np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)
    width.append(1/(dist))
    
nx.draw_networkx(g,
        pos=pos,
        nodelist=draw_nodes_names,
        edgelist=edges,
        node_size=[node_size(n) for n in draw_nodes],
        node_color=[node_color(n) for n in draw_nodes],
        labels={n.name: (n.label if n.pred is not None and n.pred > 1.0 else "") for n in draw_nodes},
        width=0.2,
        ax=ax)

# edges = g.edges()

# nx.draw_networkx_edges(g, pos=pos, arrows=False, ax=ax, width=width)

plt.show()


In [ ]:
from ete3 import Tree, TreeStyle, NodeStyle, faces, AttrFace, CircleFace

def layout(node):
    if hasattr(node, 'f'):
        frequency = float(node.f)
        lowerbound = float(node.lb)
        
        C = CircleFace(radius=frequency * 100, color="Red", style="sphere")
        C.opacity=lowerbound
        faces.add_face_to_node(C, node, 0, position="float")

    faces.add_face_to_node(AttrFace("name"), node, column=0)

# Create an empty TreeStyle
ts = TreeStyle()

# Set our custom layout function
ts.layout_fn = layout

# Draw a tree
ts.mode = "c"

# We will add node names manually
ts.show_leaf_name = True
# Show branch data
ts.show_branch_length = False
ts.show_branch_support = False


# Getting the tree
threshold = 0.03
G = C.copy(init=False)
G = G.set_predictions(preds)
print(len(G))
G = G.filter(keep_constraint = lambda n: n.pred_agg >= threshold)
print(len(G))
G.splice(keep_constraint = lambda n : len(n.children) != 1)
print(len(G))
# G.make_tree()
G.attr_label()
t = G.get_newick()
t = Tree(t, format=8, quoted_node_names=True)

# t.show(tree_style=ts)
t.render("mytree.png", w=183, units="mm", dpi=1000, tree_style=ts)
print("Done")

In [ ]:
print([
    nx.is_tree(G),
    nx.is_forest(G),
    nx.is_arborescence(G),
    nx.is_branching(G),
])

In [ ]:
T.root.children

In [ ]:
[str(c) for c in T.root.children[0].children]

In [ ]:
# Choosing two nodes
C.add_parent_store()

Is = []
for I in [0,2]:
    i = 0
    for k in C.nodes:
        n = C.nodes[k]
        if len(n.children) == 0:
            print(n)
            if i == I:
                x = n
                break
            i += 1
    Is.append(x)

In [ ]:
# two different trees
T.attr_label()

fig, axs = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={'width_ratios': [0.7, 3]})

plt.xlim([-1.3, 1.3])
draw_subgraph_to_leaf(T.nodes[Is[0].name], T, ax=axs[0])
plt.tight_layout()
# plt.savefig(f"example_{Is[0]}.pdf", dpi=600)

# plt.figure(figsize=(9,7))
# plt.xlim([-1.3, 1.3])
draw_subgraph_to_leaf(T.nodes[Is[1].name], T, ax=axs[1])
# plt.tight_layout()
plt.savefig(f"examples.pdf", dpi=600)

In [ ]:
# two different trees
P.add_parent_store()
P.attr_label()

fig, axs = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={'width_ratios': [0.7, 3]})

plt.xlim([-1.3, 1.3])
draw_subgraph_to_leaf(P.nodes[Is[0].name], T, ax=axs[0])
plt.tight_layout()
# plt.savefig(f"example_{Is[0]}.pdf", dpi=600)

# plt.figure(figsize=(9,7))
# plt.xlim([-1.3, 1.3])
draw_subgraph_to_leaf(P.nodes[Is[1].name], T, ax=axs[1])
# plt.tight_layout()
plt.savefig(f"examples.pdf", dpi=600)

In [ ]:
# two different trees
C.add_parent_store()
C.attr_label()

fig, axs = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={'width_ratios': [0.7, 3]})

plt.xlim([-1.3, 1.3])
draw_subgraph_to_leaf(C.nodes[Is[0].name], T, ax=axs[0])
plt.tight_layout()
# plt.savefig(f"example_{Is[0]}.pdf", dpi=600)

# plt.figure(figsize=(9,7))
# plt.xlim([-1.3, 1.3])
draw_subgraph_to_leaf(C.nodes[Is[1].name], T, ax=axs[1])
# plt.tight_layout()
plt.savefig(f"examples.pdf", dpi=600)

In [ ]:
T.root.children

In [ ]:
T.nodes[4224004].parents[0].parents[0].parents[0].parents[0].parents[0]

In [ ]:
for a in T.nodes[4224004].ancestors():
    print(a)

In [ ]:
# A={0,1}x[0,n]x[0,m] -> P
# 2nm places -> P

# nodes_df = pd.DataFrame({'name':[0,1,2,3,4,5],'label':[0,1,2,3,4,5]}).set_index('name')
# edges_df = pd.DataFrame({'parent':[0, 0, 0, 1, 2, 3,1],'child':[1, 2, 3, 4, 4, 4,5]})
# diamond_tail = DAG(nodes_df, edges_df)
# predictions = pd.Series([1, 1], index=[4, 5])
# t = diamond_tail.set_predictions(predictions)

# ---------- PARAMS
t = C

target_hexes = 100
kmax = 0
same_leaf_factor = 1
adjacency_factor = 1
distance_factor = 100
hexes_scaling = lambda x : x
temperature = lambda k, kmax: (1 - (k)/kmax) ** 2 / 20 #.1/np.exp(k / 100).item()
acceptance_function = lambda cost_diff, T, rv: cost_diff < 0 or np.exp((-cost_diff+0.01)/T) >= rv
quantile = .0
moved_running_avg_window = 10000
MOVED_LOWERBOUND = 0.03
# hexes_scaling = lambda x : 1 + np.sqrt(x).item() # PARAM.


# ---------- PLOTTING PARAMS
angle = - np.pi / 6
radius = 1/np.sqrt(3)
angles = np.array([angle + k * (np.pi/3) for k in range(6)])
x_hex = radius * np.cos(angles)
y_hex = radius * np.sin(angles)
x_width = np.amax(x_hex) - np.amin(x_hex)
y_width = np.amax(y_hex) - np.amin(y_hex)

def center(row, col):
    cx = (col * radius * 1.5)
    cy = np.sqrt(3) * row
    if col % 2 == 1:
        cy = cy + np.sqrt(3) * 0.5
    return cx, cy

def cartesian(a, r, c):
    return (
        a/2 + c,
        np.sqrt(3) * (a/2 + r)
    )



n = int(np.sqrt(target_hexes) // 2)
m = int(np.sqrt(target_hexes))
assert 2 * n * m <= target_hexes

scaled = [hexes_scaling(P_node.pred) for P_node in t.nodes.values() if P_node.pred is not None]
sum_scaled = sum(scaled)
to_place = []

nodes = list(t.nodes.values())
N = len(t)
indx_P_nodes, P_nodes = [], []
for i, node in enumerate(nodes):
    if (node.pred is not None and node.pred != 0):
        hexes = int(np.floor(hexes_scaling(node.pred) / sum_scaled * (2*n*m)).item())
        if hexes <= 0:
            continue
        indx_P_nodes.append(i)
        P_nodes.append(node)
        for _ in range(hexes):
            to_place.append(len(P_nodes) - 1)
P_N = len(P_nodes)


# to_place = [i for i, P_node in enumerate(P_nodes) for _ in range(int(np.floor(scaled[i] / sum_scaled * (2*n*m)).item()))]

assert 2 * n * m >= len(to_place)

t.add_parent_store()
from itertools import combinations
t.descendant_pred()
pred_nodes = t.root.preds
weights = dict()
for node in t.nodes.values():
    if node is t.root:
        continue
    significant_preds = [pred for pred in node.preds if pred in P_nodes]
    for a, b in combinations(significant_preds, 2):
        if a == b:
            assert False
        if id(a) > id(b):
            a, b = b, a
        w = weights.get((a, b), 0)
        weights[(a, b)] = w + 1 * node.depth

total_weight = sum(weights.values())

for k, v in weights.items():
    weights[k] = (v / total_weight) * adjacency_factor

# weight all adjacencies and all transitive stuff the same    
for P_node in P_nodes:
    weights[(P_node, P_node)] = same_leaf_factor / P_N

node_to_idx = {node: i for i, node in enumerate(P_nodes)}
new_weights = dict()
for k, v in weights.items():
    n1, n2 = k
    n1, n2 = node_to_idx[n1], node_to_idx[n2]
    new_weights[(n1, n2)] = v

weights = new_weights

# ------- DESIRED LOCATION --------
anc_store = t.get_ancestors()

dist = np.zeros((N, N))
for i, anc1 in enumerate(anc_store):
    for j, anc2 in enumerate(anc_store[i+1:], i+1):
#         common = anc1.intersection(anc2)
        diff = anc1.symmetric_difference(anc2)
        #d = 1/len(common)
#         d = 1/(sum(a.depth for a in common)+1)
        d = len(diff) / (len(anc1) + len(anc2))
        dist[i, j] = d
        dist[j, i] = d

similarity = dist[indx_P_nodes, :][:, indx_P_nodes]

tsne = TSNE(metric="precomputed",
            perplexity=30,
            early_exaggeration=10)
embedding = tsne.fit_transform(similarity)

x_min, y_min = cartesian(0, 0, 0)
x_max, y_max = cartesian(1, n, m)
# TODO might need to offset
x_scaler = MinMaxScaler(feature_range=(x_min,x_max))
y_scaler = MinMaxScaler(feature_range=(y_min,y_max))
embedding[:, 0] = x_scaler.fit_transform(embedding[:, 0].reshape(-1, 1)).reshape(-1)
embedding[:, 1] = y_scaler.fit_transform(embedding[:, 1].reshape(-1, 1)).reshape(-1)

# to_01_scaler = MinMaxScaler(feature_range=(0,1)).fit(embedding) # distorts distances because figure does not have asp ratio of 1

def neighbors(a, r, c):
    # interesting fact: if a and b are neighbors, and a is the x-th neighbor of b, then b is the (x+3%6)-th neighbor of a
    return (
        (a, r, (c+1) % m),
        (1-a, (r-(1-a)) % n, (c+a) % m),
        (1-a, (r-(1-a)) % n, (c-(1-a)) % m),
        (a, r, (c-1) % m),
        (1-a, (r+a) % n, (c-(1-a)) % m),
        (1-a, (r+a) % n, (c+a) % m)
    )

def cost(assignment):
    cost_matrix_adj = np.zeros((2, n, m, 6))
    it = np.nditer(assignment, flags=['multi_index'])
    for x in it:
        if x < 0:
            continue
        for neighbor_idx, neighbor in enumerate(neighbors(*it.multi_index)): # can make more eff due to symmetry 
            y = assignment[neighbor]
            if y < 0:
                continue
            w = weights.get((x.item(), y), 0)
            if w == 0:
                w = weights.get((y, x.item()), 0) # only one will exist atm
            if w != 0:
                cost_matrix_adj[(*it.multi_index, neighbor_idx)] = -w
    
    cost_matrix_dis = np.zeros((2, n, m))
    it = np.nditer(assignment, flags=['multi_index'])
    for x in it:
        if x < 0:
            continue
        tile_x, tile_y = cartesian(*it.multi_index)
        xx, xy = embedding[x].tolist()
        d = ((xx - tile_x)/(2*n-1)) ** 2 + ((xy - tile_y)/(m-1)) ** 2
        cost_matrix_dis[it.multi_index] = d
    cost_matrix_dis *= distance_factor

    cost_matrix_adj_sum = np.sum(cost_matrix_adj, axis=3)
    cost_agg = np.sum(cost_matrix_adj_sum) + np.sum(cost_matrix_dis)
    return cost_agg, cost_matrix_adj, cost_matrix_adj_sum, cost_matrix_dis

# def random_neighbor(s):
#     # SWITCHEROO
# #     x = np.random.choice(2), np.random.choice(n), np.random.choice(m)
# #     y = neighbors(*x)[np.random.choice(3)]
# #     s_new = s.copy()
# #     s_new[x], s_new[y] = s_new[y], s_new[x]
#     # RANDOM ASSIGNMENT
#     s_new = s.copy()
#     x = np.random.choice(2), np.random.choice(n), np.random.choice(m)
#     s_new[x] = np.random.choice(P_N+1) - 1
#     return s_new
     
# s = np.random.choice(P_N, replace=True, size=(2, n, m))

def cost_x(s, x):
    assignment = s[x]
    if assignment < 0:
        return [0] * 6, 0
    new_cost_x_neighbors = []
    for neighbor in neighbors(*x):
        neighbor_assignment = s[neighbor]
        if neighbor_assignment < 0:
            w = 0
        else:
            w = weights.get((assignment, neighbor_assignment), 0)
            if w == 0:
                w = weights.get((neighbor_assignment, assignment), 0)
        new_cost_x_neighbors.append(-w)

    xx, xy = cartesian(*x)
    yx, yy = embedding[assignment].tolist()
    d = ((xx - yx)/(2*n-1)) ** 2 + ((xy - yy)/(m-1)) ** 2
    d *= distance_factor
    return new_cost_x_neighbors, d

def set_cost_x(s, cost_matrix, cost_matrix_sum, x, cost_neighbors_x):
    # own
    cost_matrix[x] = cost_neighbors_x
    cost_matrix_sum[x] = np.sum(cost_matrix[x])
    # others'
    for neighbor_idx, neighbor in enumerate(neighbors(*x)):
        self_idx = (neighbor_idx + 3) % 6
        cost_matrix[(*neighbor, self_idx)] = cost_neighbors_x[neighbor_idx]
        cost_matrix_sum[neighbor] = np.sum(cost_matrix[neighbor])

def random_grid_point():
    return (np.random.choice(2), np.random.choice(n), np.random.choice(m))

# RANDOM LAYOUT
# for _ in range(2 * n * m - len(to_place)):
#     to_place.append(-1)
# s = np.array(to_place)
# np.random.shuffle(s)
# s = s.reshape(2, n, m)

# TODO better initialization - Greedy?
np.random.shuffle(to_place)
s = np.ones((2, n, m), dtype=int) * -1
# maintain a heap of: option -> max. cost reduction

def marginal_cost(s, x, assign):
    assert s[x] < 0
    s[x] = assign
    new_cost_x_neighbors, distance_cost = cost_x(s, x)
    marginal = 2 * sum(new_cost_x_neighbors) + distance_cost
    s[x] = -1
    return marginal

to_place_set = set(to_place)
options = np.zeros((2, n, m, P_N))
for a in (0,1):
    for r in range(n):
        for c in range(m):
            for assign in to_place_set:
                options[a, r, c, assign] = (
                    marginal_cost(s, (a, r, c), assign)
                )

to_place_still = {assign: to_place.count(assign) for assign in to_place_set}
to_place_still_total = len(to_place)
while to_place_still_total > 0:
    a, r, c, assign = np.unravel_index(np.argmin(options, axis=None), options.shape)
    s[(a, r, c)] = assign
    options[a, r, c, :] = np.infty
    to_place_still[assign] -= 1
    to_place_still_total -= 1
    if to_place_still_total == 0:
        break
    if to_place_still[assign] == 0:
        options[:, :, :, assign] = np.infty
        to_place_set.remove(assign)
    for neighbor in neighbors(a, r, c):
        if s[neighbor] < 0:
            for assign in to_place_set:
                if options[a, r, c, assign] != np.infty:
                    options[a, r, c, assign] = (
                        marginal_cost(s, (a, r, c), assign)
                    )  
                    
cost_s, cost_matrix_s, cost_matrix_s_sum, cost_matrix_s_distances = cost(s)
moved = 0
moved_running_avg = 0.
movable_points = None
for k in range(kmax):
    if k%1000==0:
        print(f"Cost: {np.sum(cost_matrix_s_sum) + np.sum(cost_matrix_s_distances)}. Running average % moved: {moved_running_avg}")
    T = temperature(k, kmax)
    # random neighbor testing
    if k % 10 == 0:
        moveable_points = np.arange(2*n*m)[cost_matrix_s_sum.reshape(-1) >= np.quantile(cost_matrix_s_sum, quantile)]
    x, y = np.random.choice(moveable_points, replace=False, size=2).tolist()
    x = (x//(n*m), (x//m)%n, x%m)
    y = (y//(n*m), (y//m)%n, y%m)
    if x == y: continue
    #
    old_assignment_x = s[x]
    old_assignment_y = s[y]
    old_cost_x = np.sum(cost_matrix_s[x]) + cost_matrix_s_distances[x]
    old_cost_y = np.sum(cost_matrix_s[y]) + cost_matrix_s_distances[y]
    
#     new_assignment = np.random.choice(P_N+1) - 1 # random choice of neighbor (different color in that hex)
    s[x], s[y] = old_assignment_y, old_assignment_x
    new_cost_x_neighbors, distance_cost_x = cost_x(s, x)
    new_cost_y_neighbors, distance_cost_y = cost_x(s, y)
    
    cost_diff = 2 * (
        sum(new_cost_x_neighbors) + distance_cost_x - old_cost_x
        + sum(new_cost_y_neighbors) + distance_cost_y - old_cost_y
    ) # times TWO because of double counting adjacency TODO half_neighbors?
    acceptance = acceptance_function(cost_diff, T, np.random.uniform())
    if acceptance:
        moved += 1
        just_moved = True
        set_cost_x(s, cost_matrix_s, cost_matrix_s_sum, x, new_cost_x_neighbors)
        set_cost_x(s, cost_matrix_s, cost_matrix_s_sum, y, new_cost_y_neighbors)
        cost_matrix_s_distances[x] = distance_cost_x
        cost_matrix_s_distances[y] = distance_cost_y
    else:
        just_moved = False
        s[x] = old_assignment_x
        s[y] = old_assignment_y
#     print(x, y)
    moved_running_avg = (
        (min(moved_running_avg_window-1, k) * moved_running_avg + just_moved)
        /min(moved_running_avg_window, k+1)
    )
    if moved_running_avg <= MOVED_LOWERBOUND and k > moved_running_avg_window:
        print(f"Early stop at k={k}")
        break
#     calc = cost(s)
#     assert (cost_matrix_s == calc[1]).all()
#     assert (cost_matrix_s_sum == calc[2]).all()
#     assert (cost_matrix_s_distances == calc[3]).all()
print(f"Moved {moved} out of {k} iters")

# PSEUDOCODE sim annealing
# Let s = s0
# For k = 0 through kmax (exclusive):

#     T ← temperature( 1 - (k+1)/kmax )
#     Pick a random neighbour, snew ← neighbour(s)
#     If P(E(s), E(snew), T) ≥ random(0, 1):
#         s ← snew

# Output: the final state 

assignment = s

# locations = [center(i, j) for i in range(10) for j in range(10)]

def svg_path(xc, yc, radius_modifier = 1.):
    return "M" + "L".join([str(px) + "," + str(py) for px,py in zip(xc + x_hex * radius_modifier, yc + y_hex * radius_modifier)]) + "Z"

def path(xc, yc, radius_modifier = 1.):
    x_coords = (xc + x_hex * radius_modifier).tolist()
    x_coords.append(x_coords[0])
    y_coords = (yc + y_hex * radius_modifier).tolist()
    y_coords.append(y_coords[0])
    return x_coords, y_coords


P_nodes_to_grid = {P_node: [] for P_node in P_nodes}

it = np.nditer(assignment, flags=['multi_index'])
for x in it:
    if x >= 0:
        P_nodes_to_grid[P_nodes[x]].append(it.multi_index)

t.attr_label()
depths = t.get_depths()    

colors_required = P_N + max(len(V_d) for V_d in depths)

colors = plotly.colors.sample_colorscale(colorscale='Turbo', samplepoints=colors_required)
np.random.shuffle(colors)

# max_pred = max([P_node.pred for P_node in P_nodes])


In [ ]:

fig = go.Figure()

# Update axes properties
fig.update_xaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    range=[x_min - radius, x_max + radius]
)

fig.update_yaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    scaleanchor = "x",
    scaleratio = 1,
    range=[y_min - radius, y_max + radius]
)

# ------- CREATE CONSTRAINTS --------
P_node_to_color = {P_node: color for P_node, color in zip(P_nodes, colors)} # shouldn't matter that colors is longer
d_P_node_occupancy = [{P_node: [] for P_node in P_nodes} for _ in depths]
v_to_color = dict()
for d in range(len(depths)):
    V_d = depths[d]
    # matrix of shape (V_d)
    # largest colors maintain.
    area_coverage = np.zeros(len(V_d))
    for i, v in enumerate(V_d):
        Rv = [pred for pred in v.preds if pred in P_nodes]
        coverage = []
        for P_node in Rv:
            d_P_node_occupancy[d][P_node].append(v)
            coverage += P_nodes_to_grid[P_node]
        area_coverage[i] = len(set(coverage))
    indxs = np.flip(np.argsort(area_coverage))
    
    remaining_colors = set(colors)
    for indx in indxs:
        v = V_d[indx]
        Rv = [pred for pred in v.preds if pred in P_nodes]
        max_col = -1
        max_size = -1
        for P_node in Rv:
            size = len(P_nodes_to_grid[P_node])
            col = P_node_to_color[P_node]
            if size > max_size and col in remaining_colors:
                max_size = size
                max_col = col
        if max_col == -1:
            max_col = remaining_colors.pop()
        else:
            remaining_colors.remove(max_col)
        v_color = max_col
        v_to_color[v] = v_color



disc = 100000
def simplify_grid_coords(grid_coords):
#     edges = [
#         grid_coord for grid_coord in grid_coords if not all(
#             neighbor in grid_coords for neighbor in neighbors(*grid_coord)
#         )
#     ]
#     while len(edges) != 0
    segments = dict()
    for grid_coord in grid_coords:
        cart = cartesian(*grid_coord)
        x_coords, y_coords = path(*cart)
        for i, neigh in enumerate(neighbors(*grid_coord)):
            if neigh not in grid_coords:
                x1, x2 = x_coords[i:i+2]
                y1, y2 = y_coords[i:i+2]
                x1 = int(round(x1 * disc))
                x2 = int(round(x2 * disc))
                y1 = int(round(y1 * disc))
                y2 = int(round(y2 * disc))
                adj = segments.get((x1, y1), [])
                adj.append((x2, y2))
                segments[(x1, y1)] = adj
    
    prev_point = list(segments.keys())[0]
    cycles = []
    current_cycle = [prev_point]
    while True:
        next_points = segments.get(prev_point, None)
        if next_points is None:
            cycles.append(current_cycle)
            if len(segments) == 0:
                break
            prev_point = list(segments.keys())[0]
            current_cycle = []
        elif len(next_points) == 1:
            del segments[prev_point]
            prev_point = next_points[0]
        else:
            prev_point = next_points.pop() # it matters which we pop!?
        
        current_cycle.append(prev_point)
    x_coord, y_coord = [], []
    for i, cycle in enumerate(cycles):
        if i > 0:
            x_coord.append(None)
            y_coord.append(None)
        for x_co, y_co in cycle:
            x_coord.append(x_co / disc)
            y_coord.append(y_co / disc)
    return x_coord, y_coord

v_to_trace = dict()
for d, V_d in enumerate(depths):
    for v in V_d:
        Rv = [pred for pred in v.preds if pred in P_nodes]
        x_coords = []
        y_coords = []
        for P_node in Rv:
#     color = colors[node_colors[0]]
            occupancy = d_P_node_occupancy[d][P_node]
            order = occupancy.index(v)
            radius = 1 - (order / len(occupancy))
            for j, grid_coord in enumerate(P_nodes_to_grid[P_node]):
                if j > 0:
                    x_coords.append(None)
                    y_coords.append(None)
                location = cartesian(*grid_coord)
                x_coord, y_coord = path(*location, radius_modifier = radius)
#                 if len(occupancy) == 1:
#                     pass
#                 elif len(occupancy) in [2, 3]:
#                     slices = len(occupancy)
#                     slice_size = 6 // len(occupancy)
#                     idx = order * slice_size
#                     x_coord = x_coord[idx:idx + slice_size+1]
#                     y_coord = y_coord[idx:idx + slice_size+1]
#                     x_coord.append(location[0])
#                     y_coord.append(location[1])
#                     x_coord.append(x_coord[0])
#                     y_coord.append(y_coord[0])
#                 else:
#                     raise ValueError("hi")
                x_coords.extend(x_coord)
                y_coords.extend(y_coord)
        v_to_trace[v] = len(fig.data)
        fig.add_trace(go.Scatter(x=x_coords, y=y_coords, fill="toself", mode="text", fillcolor=v_to_color[v], name=v.label, visible=False)) # mode="lines"

nodes_left = sum(len(V_d) for V_d in depths)
nodes_seen = 0  
steps = []
for d, V_d in enumerate(depths):
    nodes_this_layer = len(V_d)
    nodes_left -= nodes_this_layer
    visible = [False] * nodes_seen + [True] * nodes_this_layer + [False] * nodes_left
    nodes_seen += nodes_this_layer
    for P_node in P_nodes:
        if P_node.depth < d:
            visible[v_to_trace[P_node]] = True

    step = dict(
        method="update",
        args=[{"visible": visible},
              {"title": "Depth: " + str(i)}],  # layout attribute
    )
    steps.append(step)
        
sliders = [dict(
    active=len(depths) - 1,
    currentvalue={"prefix": "Depth: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=-0.99
    )
)

config = {'responsive': False}

# go.FigureWidget(data=fig)

fig.show(config=config)


In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.



from jupyter_dash import JupyterDash
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

app = JupyterDash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(mode="jupyterlab", debug=True)

In [ ]:
f

In [ ]:
fig

In [ ]:
simplify_grid_coords([(0, 0, 0), (0, 0, 1)])

In [ ]:
t = C
t.descendant_pred()
t.attr_label()

nodes = list(t.nodes.values())
sum_pred = sum(n.pred for n in nodes if n.pred is not None)
N = len(t)
indx_P_nodes, P_nodes = [], []
for i, node in enumerate(nodes):
    if (node.pred is not None and node.pred != 0):
        if node.pred > sum_pred / 100: # only significant stuff
            indx_P_nodes.append(i)
            P_nodes.append(node)
P_N = len(P_nodes)

depths = t.get_depths()


total_P = sum(n.pred for n in P_nodes)

def comp_coords(assignment):
    x_last = 0
    x_partition = [x_last]
    for assigned in assignment:
        pred = P_nodes[assigned].pred
        x_last += pred / total_P
        x_partition.append(x_last)
    assert abs(x_partition[-1] - 1) < 1e-6
    return x_partition

assignment = np.arange(P_N)


def cost(assignment):
    inverse_assignment = {P_nodes[j]: i for i, j in enumerate(assignment)}
    cost = 0.
    for node in nodes:
        to_draw = np.zeros(P_N+1, dtype=bool)
        for pred in node.preds:
            if pred in P_nodes:
                block = inverse_assignment[pred]
                to_draw[block] = True
        last_draw_here = None
        time_since_last_draw = 0
        for draw_here in to_draw:
            if draw_here:
                if last_draw_here is not None:
                    cost += 1 + time_since_last_draw
                last_draw_here = draw_here
                time_since_last_draw = 0
            time_since_last_draw += 1
    return cost

best = cost(assignment)
best_assignment = assignment.copy()
for _ in range(5000):
    np.random.shuffle(assignment)
    new_cost = cost(assignment)
    if new_cost < best:
        best = new_cost
        best_assignment = assignment.copy()

assignment = best_assignment
print(best)

inverse_assignment = {P_nodes[j]: i for i, j in enumerate(assignment)}
x_partition = comp_coords(assignment)
    
fig = go.Figure()

# Update axes properties
fig.update_xaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    range=[0, 1],
    fixedrange=True
)

fig.update_yaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False,
#     scaleanchor = "x",
#     scaleratio = 1,
#     range=[y_min - radius, y_max + radius]
)

def add_row(node, y=None, y_bottom=None, opacity=1., text=True):
    to_draw = np.zeros(P_N+1, dtype=bool)
    if y_bottom is None:
        y_bottom = y - (1/len(nodes))
    for pred in node.preds:
        if pred in P_nodes:
            block = inverse_assignment[pred]
            to_draw[block] = True
    x_coords, y_coords = [], []
    prev = False
    for i, draw_here in enumerate(to_draw):
        if prev == False and draw_here == True:
            if len(x_coords) > 0:
                x_coords.append(None)
                y_coords.append(None)
            x_coords.append(x_partition[i])
            x_coords.append(x_partition[i])
            y_coords.append(y)
            y_coords.append(y_bottom)
        elif prev == True and draw_here == False:
            x_coords.append(x_partition[i])
            x_coords.append(x_partition[i])
            y_coords.append(y_bottom)
            y_coords.append(y)

            x_coords.append(x_coords[len(x_coords) - 4])
            y_coords.append(y_coords[len(y_coords) - 4])
        prev = draw_here
    if text == False:
        text = ''
    else:
        text = [node.label] + ['' for _ in range(len(x_coords) - 1)]
    fig.add_trace(go.Scatter(x=x_coords, y=y_coords, fill="toself", opacity=opacity, mode="text", fillcolor=node_to_color[node], textposition="bottom right", text=text, visible=True)) # mode="lines"

y = - (len(nodes) - P_N) / len(nodes)
for V_d in depths:
    for node in V_d:
        if not (node.pred is None or node.pred == 0.):
            add_row(node, y=0, y_bottom=min(y*1.2, y-1/len(nodes)), opacity=0.1, text=False)
            add_row(node, y=y, y_bottom=min(y*1.2, y-1/len(nodes)))

y = 0
for V_d in depths:
    for node in V_d:
        if node.pred is None or node.pred == 0.:
            add_row(node, y=y)
            y -= 1 / len(nodes)
            
config = dict({'scrollZoom': True})
fig.show(config=config)


In [ ]:
t = C
t.descendant_pred()
t.add_parent_store()
t.aggregate_pred(aggregate='sum')
t.attr_label()

nodes = list(t.nodes.values())
sum_pred = sum(n.pred for n in nodes if n.pred is not None)
N = len(t)
indx_P_nodes, P_nodes = [], []
for i, node in enumerate(nodes):
    if (node.pred is not None and node.pred != 0):
        if node.pred > sum_pred / 100: # only significant stuff
            indx_P_nodes.append(i)
            P_nodes.append(node)
P_N = len(P_nodes)

depths = t.get_depths()

colors = plotly.colors.sample_colorscale(colorscale='Turbo', samplepoints=P_N)
P_node_to_color = {P_node: colors[i] for i, P_node in enumerate(P_nodes)}

node_to_color = {}
for node in nodes:
    desc_preds = node.preds
    sum_preds = sum(desc.pred for desc in desc_preds)
    rgb = 0.
    for desc in desc_preds:
        desc_color = np.array([x for x in map(int, re.findall(r'\d+', P_node_to_color[desc]))])
        rgb += (1/len(desc_preds)) * np.square(desc_color)
    node_to_color[node] = 'rgb(' + ','.join([str(x) for x in np.sqrt(rgb)]) + ')'
#     max_pred = -1
#     desc = -1
#     for desc_pred in desc_preds:
#         if desc_pred.pred > max_pred:
#             max_pred = desc_pred.pred
#             desc = desc_pred
#     node_to_color[node] = P_node_to_color[desc]

source, target, value, color = [], [], [], []

node_to_index = {node: i for i, node in enumerate(nodes)}
for i, n in enumerate(nodes):
    for c in n.children:
        source.append(i)
        target.append(node_to_index[c])
        width = c.pred_agg
        width /= len(c.parents)
        value.append(width)
        color.append(node_to_color[n])

        

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = [n.label for n in nodes],
      color = [node_to_color[n] for n in nodes]
    ),
    link = dict(
      source = source, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target,
      value = value,
      color = color,
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [ ]:
import re
np.array([x for x in map(int, re.findall(r'\d+', colors[0]))])

In [ ]:
embedding = np.array([[0,0],[0,1],[1,1]])
N_points = len(embedding)
# move points away from eachother
pw = pairwise_distances(embedding)
for i in range(N_points):
    pw[i,i] = np.inf
indx = (pw == 0).nonzero()
for i, j in zip(*indx):
    pw[i,j] = np.inf
F = np.divide(0.1, pw * pw)
assert F.shape == pw.shape 
move = np.tile(embedding.reshape(N_points, 1, 2), (1, N_points, 1)) - np.tile(embedding.reshape(1, N_points, 2), (N_points, 1, 1))
displacement = np.tile(F.reshape(N_points, N_points, 1), (1, 1, 2)) * move
new = embedding + np.sum(displacement, axis=1)
print(embedding)
print(new)

In [ ]:
np.isnan(new).any()

In [ ]:
embedding[[44, 130]]

In [ ]:
embedding

In [ ]:
plt.scatter(x=embedding[:, 0], y = embedding[:, 1])
for i, lab in enumerate([n.label for n in P_nodes]):
    plt.annotate(lab, (embedding[i, 0], embedding[i, 1]))

In [ ]:
similarity

In [ ]:
regions[0]


In [ ]:


min_x = -max_norm_embedding
min_y = -max_norm_embedding
max_x = max_norm_embedding
max_y = max_norm_embedding
box = Polygon([[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]])

# colorize
for region in regions:
#     polygon = vertices[region]
    # Clipping polygon
#     poly = Polygon(polygon)
    poly = region.intersection(box)
    polygon = [p for p in poly.exterior.coords]

    plt.fill(*zip(*polygon), alpha=0.4)

In [ ]:

min_x = -max_norm_embedding
min_y = -max_norm_embedding
max_x = max_norm_embedding
max_y = max_norm_embedding
box = Polygon([[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]])

# VORONOI COMPUTE
points = MultiPoint(embedding.tolist())

regions = voronoi_diagram(points, envelope=box)

# colorize
for region in regions:
#     polygon = vertices[region]
    # Clipping polygon
#     poly = Polygon(polygon)
    poly = region.intersection(box)
    polygon = [p for p in poly.exterior.coords]
    plt.fill(*zip(*polygon))


In [ ]:
# colorize
for region in regions:
#     polygon = vertices[region]
    # Clipping polygon
#     poly = Polygon(polygon)
    poly = region.intersection(box)
    polygon = [p for p in poly.exterior.coords]
    plt.fill(*zip(*polygon), alpha=0.4)


In [ ]:
F_edge = np.zeros(embedding.shape)
magnitude = np.linalg.norm(embedding, ord=5, axis=1)
F_edge[:, 0] = (-embedding[:, 0]) * magnitude
F_edge[:, 1] = (-embedding[:, 1]) * magnitude

In [ ]:
np.mean(np.linalg.norm(F_edge, axis=1))

In [ ]:
weights = np.array([p.pred for p in P])
cross = weights.reshape(-1, 1)@weights.reshape(1, -1)
weights.shape

In [ ]:
cross = weights.reshape(-1, 1)@weights.reshape(1, -1)

In [ ]:
cross.shape

In [ ]:
cross[5,4] == weights[5] * weights[4]

In [ ]:
test = np.arange(1000).reshape(10, 10, 10)
x = (1, 2)
test[x]


In [ ]:
for x in np.arange(2*n*m)[np.sum(cost_matrix_s, axis=3).reshape(-1) == 0]:
    coord = (x//(n*m), (x//m)%n, x%m)
    print(np.sum(cost_matrix_s, axis=3)[coord])

In [ ]:
cost_matrix_s_sum

In [ ]:
np.unravel_index(np.argmin(cost_matrix_s_sum, axis=None), cost_matrix_s_sum.shape)

In [ ]:
np.sum(options == np.min(options, axis=None))

In [ ]:
x = (dict(a=1))

In [ ]:
del x['a']
x